<b>Agenda</b>
1) Using pipelines
<br>
2) Construct plots
<br>
3) Improve performance
<br>
4) Push code into Github
<br>
<br>
<b>Long-term goals</b>
1) Loading data in batches
<br>
2) Parallel processing (perhaps using Apache Spark)

In [1]:
import pandas as pd
import tensorflow as tf
import os
import numpy as np
import math
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow import keras
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
dataset_df = pd.read_csv('train.csv')

# 1) View summary of the data

In [3]:
print(f'Shape of the training data = ', dataset_df.shape)

Shape of the training data =  (5237980, 17)


In [4]:
print('First 5 rows: -')
dataset_df.head()

First 5 rows: -


,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
0,0,0,0,3180602.69,1,0.999812,13380276.64,NaN,NaN,0.999812,60651.50,1.000026,8493.03,1.0,-3.029704,0,0_0_0
1,1,0,0,166603.91,-1,0.999896,1642214.25,NaN,NaN,0.999896,3233.04,1.000660,20605.09,1.0,-5.519986,0,0_0_1
2,2,0,0,302879.87,-1,0.999561,1819368.03,NaN,NaN,0.999403,37956.00,1.000298,18995.00,1.0,-8.389950,0,0_0_2
3,3,0,0,11917682.27,-1,1.000171,18389745.62,NaN,NaN,0.999999,2324.90,1.000214,479032.40,1.0,-4.010200,0,0_0_3
4,4,0,0,447549.96,-1,0.999532,17860614.95,NaN,NaN,0.999394,16485.54,1.000016,434.10,1.0,-7.349849,0,0_0_4


In [5]:
print('Description of the data: -')
dataset_df.describe()

Description of the data: -


,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id
count,5.237980e+06,5.237980e+06,5.237980e+06,5.237760e+06,5.237980e+06,5.237760e+06,5.237760e+06,2.343638e+06,2.380800e+06,5.237760e+06,5.237980e+06,5.237760e+06,5.237980e+06,5.237760e+06,5.237892e+06,5.237980e+06
mean,9.928856e+01,2.415100e+02,2.700000e+02,5.715293e+06,-1.189619e-02,9.999955e-01,4.510025e+07,1.001713e+00,9.996601e-01,9.997263e-01,5.181359e+04,1.000264e+00,5.357568e+04,9.999920e-01,-4.756125e-02,1.331005e+04
std,5.787176e+01,1.385319e+02,1.587451e+02,2.051591e+07,8.853374e-01,2.532497e-03,1.398413e+08,7.214705e-01,1.216920e-02,2.499345e-03,1.114214e+05,2.510042e-03,1.293554e+05,2.497509e-03,9.452860e+00,7.619271e+03
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,9.352850e-01,4.316610e+03,7.700000e-05,7.869880e-01,9.349150e-01,0.000000e+00,9.398270e-01,0.000000e+00,9.380080e-01,-3.852898e+02,0.000000e+00
25%,4.900000e+01,1.220000e+02,1.300000e+02,8.453415e+04,-1.000000e+00,9.987630e-01,5.279575e+06,9.963320e-01,9.971000e-01,9.985290e-01,7.374720e+03,9.990290e-01,7.823700e+03,9.987810e-01,-4.559755e+00,6.729000e+03
50%,9.900000e+01,2.420000e+02,2.700000e+02,1.113604e+06,0.000000e+00,9.999670e-01,1.288264e+07,9.998830e-01,9.998890e-01,9.997280e-01,2.196900e+04,1.000207e+00,2.301792e+04,9.999970e-01,-6.020069e-02,1.334500e+04
75%,1.490000e+02,3.610000e+02,4.100000e+02,4.190951e+06,1.000000e+00,1.001174e+00,3.270013e+07,1.003318e+00,1.002590e+00,1.000905e+00,5.583168e+04,1.001414e+00,5.787841e+04,1.001149e+00,4.409552e+00,1.990700e+04
max,1.990000e+02,4.800000e+02,5.400000e+02,2.982028e+09,1.000000e+00,1.077488e+00,7.713682e+09,4.379531e+02,1.309732e+00,1.077488e+00,3.028784e+07,1.077836e+00,5.440500e+07,1.077675e+00,4.460704e+02,2.645400e+04


# 2) Handling missing values

## 2.1) Determine missingness in the dataframe

In [6]:
print("List of missing values for various columns of the dataframe: -")
num_missing_vals_series = (dataset_df.isnull().sum(axis = 0)/dataset_df.shape[0]) * 100
num_missing_vals_series

List of missing values for various columns of the dataframe: -


stock_id                    0.000000
date_id                     0.000000
seconds_in_bucket           0.000000
imbalance_size              0.004200
imbalance_buy_sell_flag     0.000000
reference_price             0.004200
matched_size                0.004200
far_price                  55.256836
near_price                 54.547364
bid_price                   0.004200
bid_size                    0.000000
ask_price                   0.004200
ask_size                    0.000000
wap                         0.004200
target                      0.001680
time_id                     0.000000
row_id                      0.000000
dtype: float64

In [7]:
index = 0

for feature in num_missing_vals_series.index:
    
    if (num_missing_vals_series[feature] > 0) and (num_missing_vals_series[feature] < 1):
        print(f'Feature = {feature}')
        print(f'Percentage of missing values = {num_missing_vals_series[feature]}')
        print(f'Dropping missing values for {feature}')
        dataset_df.dropna(subset = feature, inplace = True)
        print('\n')
    
    index += 1

Feature = imbalance_size
Percentage of missing values = 0.004200092402032845
Dropping missing values for imbalance_size


Feature = reference_price
Percentage of missing values = 0.004200092402032845
Dropping missing values for reference_price


Feature = matched_size
Percentage of missing values = 0.004200092402032845
Dropping missing values for matched_size


Feature = bid_price
Percentage of missing values = 0.004200092402032845
Dropping missing values for bid_price


Feature = ask_price
Percentage of missing values = 0.004200092402032845
Dropping missing values for ask_price


Feature = wap
Percentage of missing values = 0.004200092402032845
Dropping missing values for wap


Feature = target
Percentage of missing values = 0.0016800369608131378
Dropping missing values for target




In [8]:
num_missing_vals_series = (dataset_df.isnull().sum(axis = 0)/dataset_df.shape[0]) * 100
num_missing_vals_series

stock_id                    0.000000
date_id                     0.000000
seconds_in_bucket           0.000000
imbalance_size              0.000000
imbalance_buy_sell_flag     0.000000
reference_price             0.000000
matched_size                0.000000
far_price                  55.254956
near_price                 54.545455
bid_price                   0.000000
bid_size                    0.000000
ask_price                   0.000000
ask_size                    0.000000
wap                         0.000000
target                      0.000000
time_id                     0.000000
row_id                      0.000000
dtype: float64

In [9]:
dataset_df.fillna(0, inplace = True)

In [10]:
num_missing_vals_series = (dataset_df.isnull().sum(axis = 0)/dataset_df.shape[0]) * 100
num_missing_vals_series

stock_id                   0.0
date_id                    0.0
seconds_in_bucket          0.0
imbalance_size             0.0
imbalance_buy_sell_flag    0.0
reference_price            0.0
matched_size               0.0
far_price                  0.0
near_price                 0.0
bid_price                  0.0
bid_size                   0.0
ask_price                  0.0
ask_size                   0.0
wap                        0.0
target                     0.0
time_id                    0.0
row_id                     0.0
dtype: float64

# 3) Reshaping and preparing data

In [11]:
dataset_reshaped_df = dataset_df
dataset_reshaped_df.sort_values(by = ['stock_id', 'date_id', 'seconds_in_bucket'], inplace = True)

In [12]:
def compute_features(df):
    
    df['imbalance_size'] = df['imbalance_size'] * df['imbalance_buy_sell_flag']

def compute_rollover_features(df):
    
    df['prev_target'] = df.groupby('stock_id')['target'].shift(1)
    df.fillna(0, inplace = True)
    
    return df

def standardize_features(df, mu_dict=None, sigma_dict=None):
    
    if (mu_dict is None) and (sigma_dict is None):
        
        mu_dict = {}
        sigma_dict = {}
        
        mu_dict['imbalance_size'] = [0]*200
        mu_dict['reference_price'] = [0]*200
        mu_dict['matched_size'] = [0]*200
        mu_dict['far_price'] = [0]*200
        mu_dict['near_price'] = [0]*200
        mu_dict['bid_price'] = [0]*200
        mu_dict['bid_size'] = [0]*200
        mu_dict['ask_price'] = [0]*200
        mu_dict['ask_size'] = [0]*200
        mu_dict['wap'] = [0]*200
        mu_dict['prev_target'] = [0]*200
        
        sigma_dict['imbalance_size'] = [0]*200
        sigma_dict['reference_price'] = [0]*200
        sigma_dict['matched_size'] = [0]*200
        sigma_dict['far_price'] = [0]*200
        sigma_dict['near_price'] = [0]*200
        sigma_dict['bid_price'] = [0]*200
        sigma_dict['bid_size'] = [0]*200
        sigma_dict['ask_price'] = [0]*200
        sigma_dict['ask_size'] = [0]*200
        sigma_dict['wap'] = [0]*200
        sigma_dict['prev_target'] = [0]*200
        
        for stock_id in range(df['stock_id'].max()+1):
            
            mu_dict['imbalance_size'][stock_id] = df[df['stock_id'] == stock_id]['imbalance_size'].mean()
            mu_dict['reference_price'][stock_id] = df[df['stock_id'] == stock_id]['reference_price'].mean()
            mu_dict['matched_size'][stock_id] = df[df['stock_id'] == stock_id]['matched_size'].mean()
            mu_dict['far_price'][stock_id] = df[df['stock_id'] == stock_id]['far_price'].mean()
            mu_dict['near_price'][stock_id] = df[df['stock_id'] == stock_id]['near_price'].mean()
            mu_dict['bid_price'][stock_id] = df[df['stock_id'] == stock_id]['bid_price'].mean()
            mu_dict['bid_size'][stock_id] = df[df['stock_id'] == stock_id]['bid_size'].mean()
            mu_dict['ask_price'][stock_id] = df[df['stock_id'] == stock_id]['ask_price'].mean()
            mu_dict['ask_size'][stock_id] = df[df['stock_id'] == stock_id]['ask_size'].mean()
            mu_dict['wap'][stock_id] = df[df['stock_id'] == stock_id]['wap'].mean()
            mu_dict['prev_target'][stock_id] = df[df['stock_id'] == stock_id]['prev_target'].mean()
            
            sigma_dict['imbalance_size'][stock_id] = df[df['stock_id'] == stock_id]['imbalance_size'].std()
            sigma_dict['reference_price'][stock_id] = df[df['stock_id'] == stock_id]['reference_price'].std()
            sigma_dict['matched_size'][stock_id] = df[df['stock_id'] == stock_id]['matched_size'].std()
            sigma_dict['far_price'][stock_id] = df[df['stock_id'] == stock_id]['far_price'].std()
            sigma_dict['near_price'][stock_id] = df[df['stock_id'] == stock_id]['near_price'].std()
            sigma_dict['bid_price'][stock_id] = df[df['stock_id'] == stock_id]['bid_price'].std()
            sigma_dict['bid_size'][stock_id] = df[df['stock_id'] == stock_id]['bid_size'].std()
            sigma_dict['ask_price'][stock_id] = df[df['stock_id'] == stock_id]['ask_price'].std()
            sigma_dict['ask_size'][stock_id] = df[df['stock_id'] == stock_id]['ask_size'].std()
            sigma_dict['wap'][stock_id] = df[df['stock_id'] == stock_id]['wap'].std()
            sigma_dict['prev_target'][stock_id] = df[df['stock_id'] == stock_id]['prev_target'].std()
    
    df['imbalance_size'] = (df['imbalance_size'] - df['stock_id'].map(lambda x: mu_dict['imbalance_size'][x]))/df['stock_id'].map(lambda x: sigma_dict['imbalance_size'][x])
    df['reference_price'] = (df['reference_price'] - df['stock_id'].map(lambda x: mu_dict['reference_price'][x]))/df['stock_id'].map(lambda x: sigma_dict['reference_price'][x])
    df['matched_size'] = (df['matched_size'] - df['stock_id'].map(lambda x: mu_dict['matched_size'][x]))/df['stock_id'].map(lambda x: sigma_dict['matched_size'][x])
    df['far_price'] = (df['far_price'] - df['stock_id'].map(lambda x: mu_dict['far_price'][x]))/df['stock_id'].map(lambda x: sigma_dict['far_price'][x])
    df['near_price'] = (df['near_price'] - df['stock_id'].map(lambda x: mu_dict['near_price'][x]))/df['stock_id'].map(lambda x: sigma_dict['near_price'][x])
    df['bid_price'] = (df['bid_price'] - df['stock_id'].map(lambda x: mu_dict['bid_price'][x]))/df['stock_id'].map(lambda x: sigma_dict['bid_price'][x])
    df['bid_size'] = (df['bid_size'] - df['stock_id'].map(lambda x: mu_dict['bid_size'][x]))/df['stock_id'].map(lambda x: sigma_dict['bid_size'][x])
    df['ask_price'] = (df['ask_price'] - df['stock_id'].map(lambda x: mu_dict['ask_price'][x]))/df['stock_id'].map(lambda x: sigma_dict['ask_price'][x])
    df['ask_size'] = (df['ask_size'] - df['stock_id'].map(lambda x: mu_dict['ask_size'][x]))/df['stock_id'].map(lambda x: sigma_dict['ask_size'][x])    
    df['wap'] = (df['wap'] - df['stock_id'].map(lambda x: mu_dict['wap'][x]))/df['stock_id'].map(lambda x: sigma_dict['wap'][x])        
        
    df['prev_target'] = (df['prev_target'] - df['stock_id'].map(lambda x: mu_dict['prev_target'][x]))/df['stock_id'].map(lambda x: sigma_dict['prev_target'][x])
    
    return (mu_dict, sigma_dict)
   
def drop_features(df):
    
    #Drop the following features.
    df.drop(['row_id', 'imbalance_buy_sell_flag'], axis = 1, inplace = True)
    
    if 'currently_scored' in df.columns:
        df.drop(['currently_scored'], axis = 1, inplace = True)

In [13]:
#Recompute some features.
compute_features(dataset_reshaped_df)

#Compute roll-over features.
dataset_reshaped_df = compute_rollover_features(dataset_reshaped_df)

#Standardize features.
mu_dict, sigma_dict = standardize_features(dataset_reshaped_df)

#Drop features from the dataset.
drop_features(dataset_reshaped_df)

In [14]:
dataset_reshaped_df[dataset_reshaped_df['stock_id'] == 0].head()

,stock_id,date_id,seconds_in_bucket,imbalance_size,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,prev_target
0,0,0,0,0.420313,-0.013518,-0.587909,-0.906572,-0.912704,0.045702,0.326687,0.042624,-0.517844,0.094030,-3.029704,0,0.043116
191,0,0,10,0.218021,0.113973,-0.498553,-0.906572,-0.912704,0.045702,-0.292750,0.042624,-0.257284,0.029652,0.389814,1,-0.457311
382,0,0,20,0.218021,0.050228,-0.498553,-0.906572,-0.912704,0.045702,-0.416637,-0.021117,-0.454750,-0.000153,4.220009,2,0.107503
573,0,0,30,0.218021,0.177718,-0.498553,-0.906572,-0.912704,0.173495,0.264903,0.106365,0.136070,0.144697,5.450249,3,0.740151
764,0,0,40,0.209248,0.369550,-0.494678,-0.906572,-0.912704,0.301885,-0.283994,0.298183,-0.203679,0.282991,3.169775,4,0.943354


In [15]:
train_df = dataset_reshaped_df[dataset_reshaped_df['date_id'] < 478]
val_df = dataset_reshaped_df[dataset_reshaped_df['date_id'] >= 478]

In [16]:
def get_similar_training_data(stock_id, test_val_data_dates, threshold = 0.9):
    
    #Get all training data associated with the stock_id.
    stock_train_df = train_df[train_df['stock_id'] == stock_id].copy()
    
    #Drop columns not relevant for the cosine similarity and convert the result to arrays.
    train_curr_date_array = stock_train_df.drop(['stock_id', 'target', 'date_id', 'time_id', 'seconds_in_bucket'], axis = 1).values
    train_target_date_array = (stock_train_df['target'].values).reshape(-1, 1)
    
    indices_of_closest_arrays = None
    
    for test_val_data_date in test_val_data_dates:
        
        #Compute the cosine similarity between the single row of test data and the all the rows of training data.
        cos_sim = cosine_similarity(train_curr_date_array, test_val_data_date.reshape(1, -1))
        
        if indices_of_closest_arrays is None:
            #Determine the indices of training data with higher cosine similarity than the threshold.
            indices_of_closest_arrays = np.where(cos_sim > threshold)[0]
        
        else:
            indices_of_closest_arrays = np.concatenate((indices_of_closest_arrays, np.where(cos_sim > threshold)[0]))
    
    indices_of_closest_arrays = np.unique(indices_of_closest_arrays)
    indices_of_closest_arrays = np.sort(indices_of_closest_arrays)
    
    #Get training rows corressponding to the indices determined earlier.
    ret_train_arr = stock_train_df.iloc[indices_of_closest_arrays].values
    ret_train_arr = ret_train_arr.reshape(1, ret_train_arr.shape[0], ret_train_arr.shape[1])
    
    return ret_train_arr

#& (val_df['seconds_in_bucket'] == 0)
tr_array = get_similar_training_data(2, val_df[(val_df['stock_id'] == 2) & (val_df['date_id'] == 478)].drop(['stock_id', 'target', 'date_id', 'time_id', 'seconds_in_bucket', 'target'], axis = 1).values, threshold = 0.9)
print(tr_array.shape)

(1, 7746, 16)


In [17]:
def get_valid_test_data(stock_id):
    
    test_val_df = val_df[val_df['stock_id'] == stock_id].copy()
    #test_val_df.drop(['stock_id', 'date_id', 'time_id', 'seconds_in_bucket'], inplace = True, axis = 1)
    
    val_target_array = None
    
    if 'target' in test_val_df.columns:
        val_target_array = (test_val_df['target'].values).reshape(-1, 1)
        test_val_df.drop(['target'], inplace = True, axis = 1)
    
    val_array = test_val_df.values
    val_target_array = val_target_array
    
    return val_array, val_target_array

tr_array = get_valid_test_data(0)
print(tr_array[0].shape, tr_array[0].shape)

(165, 15) (165, 15)


# 4) Creating models specific to each stock ID

In [18]:
#Function declarations
def step_decay(epoch, learning_rate):
    #Initialize the base initial learning rate, drop factor, and epochs to drop every
    init_lr = 1
    factor = 0.9
    drop_every = 6
    # compute learning rate for the current epoch
    learning_rate = init_lr*(factor ** (np.floor(epoch) / drop_every))
    return learning_rate

In [19]:
val_df[(val_df['stock_id'] == 0) & (val_df['time_id'] == 26290)].values

array([[ 0.00000000e+00,  4.78000000e+02,  0.00000000e+00,
        -3.25478308e-01,  2.40145801e-02, -6.74912495e-01,
        -9.06572064e-01, -9.12704449e-01,  8.33233301e-02,
        -1.74007138e-01,  5.69211617e-02, -5.05973453e-01,
         9.40295844e-02, -5.42998300e+00,  2.62900000e+04,
         5.11418658e-03]])

In [20]:
#Input dimension - 1 has been added because 'PREV_TARGET field is introduced much later'
INPUT_SIZE = 11
print(f'Input size = {INPUT_SIZE}')

#Declaring a dictionary of models - a model for each stock ID.
models_dict = {}

#Sequence length of the model.
SEQ_LEN = 55 * 1
print(f'Sequence length = {SEQ_LEN}')

Input size = 11
Sequence length = 55


In [21]:
#Get sequence of a specific length starting backwards from the records in data_array.
def get_sequence(seq_len, curr_stock_id, data_array):
        
    #Create an empty array of sequences
    data_seq_array = np.zeros((data_array.shape[0], seq_len, data_array.shape[1]))
    
    #Fetch all training data associated with the current stock ID.
    stock_train_array = train_df[train_df['stock_id'] == curr_stock_id].values
    
    #Iterate through the rows of the validation data.
    for index, row in enumerate(data_array):
        
        #print('-', index)
        
        #print(stock_train_array[:, -2].shape, row[-2])
        #print(stock_train_array[:, -2] >= row[-2] - seq_len + 1) & (stock_train_array[:, -2] <= row[-2])
        
        
        #Get the sequence associated with the current row.
        cond_mask = (stock_train_array[:, -2] >= row[-2] - seq_len + 1) & (stock_train_array[:, -2] <= row[-2])
        
        #Get the number of sequence elements found in the training data.
        num_seq_elem_train = np.sum(cond_mask)
        #print("1)", num_seq_elem_train)
        
        #Fill up the target array with the rows found in the training data array.
        data_seq_array[index][0:num_seq_elem_train] = stock_train_array[cond_mask][0:num_seq_elem_train]
        #print("2)", stock_train_array[cond_mask][0:num_seq_elem_train].shape)
        
        #Fill up the target array with rows found in the validationd data array (not found in training data array)
        data_seq_array[index][num_seq_elem_train:] = data_array[0:seq_len-num_seq_elem_train]
        #print("3)", data_array[0:seq_len-num_seq_elem_train].shape)
    
    return data_seq_array
        
#test_array = get_sequence(SEQ_LEN, 0)
#print(test_array.shape)
#print(test_array[0])

<b><u>Note: -</u></b><br>
It was earlier found that the training error in most cases was higher than the validation error.<br>
<br>
<b><u>Conjecture: -</u></b><br>
The training data is selected on the basis of its cosine similarity to the validation data. Therefore, the records comprising the training data are more similar to the validation data than they are to each other. This causes the validation error to be lower than the training error.<br>
<br>
<b><u>Conclusion: -</u></b><br>
Because of the manner of selecting training data, the training error is likely to be higher than the validation error. Therefore, training error does not mean much here. The efficacy of the model will be determined purely based on the validation error.

In [22]:
def train_models(threshold):
    
    early_checkpoint = EarlyStopping(patience=2, monitor='mae', mode='min')

    seed_value = 42
    tf.random.set_seed(seed_value)

    #Construct train/validation sets separately for all stock IDs.
    for curr_stock_id in dataset_reshaped_df['stock_id'].unique():
        
        print(f'Training model for stock ID {curr_stock_id}')
        
        curr_train_mae = 0
        curr_val_mae = 0
        
        val_data_array = val_df[val_df['stock_id'] == curr_stock_id].copy().values
        
        #Get SEQ_LEN records prior to the validation rows for the current stock ID.
        val_seq_array = get_sequence(SEQ_LEN, curr_stock_id, val_data_array)
        #print('Validation sequence: - ', val_seq_array.shape)
        
        #For each sequence, select training records that are similar to the validation records (last element of the 
        #validation sequence) determined above. The following indices will be removed while determining similarity: -
        #'stock_id' - 1
        #'date_id' - 2
        #'seconds_in_bucket' - 3
        #'target' - 14
        #'time_id' - 15
        excluded_indices = np.array([1,2,3,14,15])
        indices_mask = np.ones(val_seq_array.shape[2], dtype = bool)
        indices_mask[excluded_indices] = False
        curr_train_similar_rows = get_similar_training_data(curr_stock_id, val_seq_array[:, -1, indices_mask], threshold = threshold)
        print(curr_train_similar_rows.shape)
        if curr_train_similar_rows.shape[1] <= 1000:
            curr_train_similar_rows = get_similar_training_data(curr_stock_id, val_seq_array[:, -1, indices_mask], threshold = threshold - 0.10)
            if curr_train_similar_rows.shape[1] <= 1000:
                curr_train_similar_rows = get_similar_training_data(curr_stock_id, val_seq_array[:, -1, indices_mask], threshold = threshold - 0.20)
                if curr_train_similar_rows.shape[1] <= 1000:
                    curr_train_similar_rows = get_similar_training_data(curr_stock_id, val_seq_array[:, -1, indices_mask], threshold = threshold - 0.30)
                    if curr_train_similar_rows.shape[1] <= 1000:
                        curr_train_similar_rows = get_similar_training_data(curr_stock_id, val_seq_array[:, -1, indices_mask], threshold = threshold - 0.40)
                        if curr_train_similar_rows.shape[1] <= 1000:
                            curr_train_similar_rows = get_similar_training_data(curr_stock_id, val_seq_array[:, -1, indices_mask], threshold = threshold - 0.50)
                            print(f"Threshold = {threshold - 0.50}")
                        else:
                            print(f"Threshold = {threshold - 0.40}")
                    else:
                        print(f"Threshold = {threshold - 0.30}")
                else:
                    print(f"Threshold = {threshold - 0.20}")
            else:
                print(f"Threshold = {threshold - 0.10}")
        else:
            print(f"Threshold = {threshold}")
        
        #print("Similar training rows' shape: - ", curr_train_similar_rows.shape)
        
        #Get SEQ_LEN records prior to the training rows for the current stock ID.
        train_seq_array = get_sequence(SEQ_LEN, curr_stock_id, curr_train_similar_rows[0])
        #print('Training sequence: - ', train_seq_array.shape)
        
        #Construct a GRU model for the current stock ID.
        model = tf.keras.models.Sequential([
            tf.keras.layers.GRU(8, return_sequences=True, input_shape=(SEQ_LEN, INPUT_SIZE)),
            tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(1, activation='linear'))])
        
        #Compile the model.
        lr_scheduler = keras.callbacks.LearningRateScheduler(step_decay)
        opt = tf.keras.optimizers.Adam(learning_rate=0.01)
        model.compile(loss="mean_absolute_error", optimizer=opt, metrics=["mae"])
        
        #Create training dataset.
        print('Training data set size = ', train_seq_array.shape)
        train_dataset = tf.data.Dataset.from_tensor_slices((train_seq_array[:, :, indices_mask], train_seq_array[:, :, 15]))
        train_dataset = train_dataset.batch(50)
        
        #Create validation dataset.
        print('Validation data set size = ', val_seq_array.shape)
        val_dataset = tf.data.Dataset.from_tensor_slices((val_seq_array[:, :, indices_mask], val_seq_array[:, :, 15]))
        val_dataset = val_dataset.batch(50)
        
        #Train the model.
        history = model.fit(train_dataset, validation_data = val_dataset, epochs=20, callbacks=[early_checkpoint, lr_scheduler], verbose = 0)
            
        #Determine the cumulative training and validation MAEs across the different rows of the validation
        #data.
        curr_train_mae = history.history['mae'][-1]
        curr_val_mae = history.history['val_mae'][-1]
        
        print(f'Training MAE = {curr_train_mae}, Validation MAE = {curr_val_mae}')
        
        if curr_stock_id not in models_dict.keys():
            models_dict[curr_stock_id] = [model, threshold, curr_train_mae, curr_val_mae]
        
        else:
            if models_dict[curr_stock_id][3] > curr_val_mae:
                models_dict[curr_stock_id][0] = model
                models_dict[curr_stock_id][1] = threshold   
                models_dict[curr_stock_id][2] = curr_train_mae
                models_dict[curr_stock_id][3] = curr_val_mae
        
        print('------------------------------')

In [23]:
%%time

#threshold_list = [0.95, 0.9, 0.8, 0.7] 
threshold_list = [0.95]

for threshold in threshold_list:

    print('Cosine similarity threshold = ', threshold)
    train_models(threshold)

Cosine similarity threshold =  0.95
Training model for stock ID 0
(1, 2199, 16)
Threshold = 0.95
Training data set size =  (2199, 55, 16)
Validation data set size =  (165, 55, 16)
Training MAE = 0.7623547911643982, Validation MAE = 0.7081812620162964
------------------------------
Training model for stock ID 1
(1, 166, 16)
Threshold = 0.85
Training data set size =  (1778, 55, 16)
Validation data set size =  (165, 55, 16)
Training MAE = 0.7468001246452332, Validation MAE = 0.6311120986938477
------------------------------
Training model for stock ID 2
(1, 20, 16)
Threshold = 0.75
Training data set size =  (4601, 55, 16)
Validation data set size =  (165, 55, 16)
Training MAE = 0.8455199599266052, Validation MAE = 0.8830938339233398
------------------------------
Training model for stock ID 3
(1, 220, 16)
Threshold = 0.85
Training data set size =  (1427, 55, 16)
Validation data set size =  (165, 55, 16)
Training MAE = 0.7316550612449646, Validation MAE = 0.5541003346443176
---------------

Training data set size =  (1504, 55, 16)
Validation data set size =  (165, 55, 16)
Training MAE = 0.9449955821037292, Validation MAE = 0.915200412273407
------------------------------
Training model for stock ID 33
(1, 30, 16)
Threshold = 0.5499999999999999
Training data set size =  (1315, 55, 16)
Validation data set size =  (165, 55, 16)
Training MAE = 0.8166882395744324, Validation MAE = 0.5693486332893372
------------------------------
Training model for stock ID 34
(1, 19, 16)
Threshold = 0.5499999999999999
Training data set size =  (1732, 55, 16)
Validation data set size =  (165, 55, 16)
Training MAE = 0.8817002773284912, Validation MAE = 0.8273182511329651
------------------------------
Training model for stock ID 35
(1, 40, 16)
Threshold = 0.5499999999999999
Training data set size =  (1481, 55, 16)
Validation data set size =  (165, 55, 16)
Training MAE = 0.8040090799331665, Validation MAE = 0.5831952095031738
------------------------------
Training model for stock ID 36
(1, 30, 

(1, 21, 16)
Threshold = 0.44999999999999996
Training data set size =  (914, 55, 16)
Validation data set size =  (165, 55, 16)
Training MAE = 0.7397549152374268, Validation MAE = 0.6951972246170044
------------------------------
Training model for stock ID 65
(1, 10, 16)
Threshold = 0.44999999999999996
Training data set size =  (1322, 55, 16)
Validation data set size =  (165, 55, 16)
Training MAE = 0.870607852935791, Validation MAE = 0.7087659239768982
------------------------------
Training model for stock ID 66
(1, 42, 16)
Threshold = 0.44999999999999996
Training data set size =  (819, 55, 16)
Validation data set size =  (165, 55, 16)
Training MAE = 0.8672786951065063, Validation MAE = 0.9588919878005981
------------------------------
Training model for stock ID 67
(1, 30, 16)
Threshold = 0.44999999999999996
Training data set size =  (1468, 55, 16)
Validation data set size =  (165, 55, 16)
Training MAE = 0.9225841164588928, Validation MAE = 0.8200339674949646
-------------------------

(1, 68, 16)
Threshold = 0.5499999999999999
Training data set size =  (1049, 55, 16)
Validation data set size =  (165, 55, 16)
Training MAE = 0.9378141760826111, Validation MAE = 0.7528418898582458
------------------------------
Training model for stock ID 97
(1, 36, 16)
Threshold = 0.5499999999999999
Training data set size =  (1136, 55, 16)
Validation data set size =  (165, 55, 16)
Training MAE = 0.9095420241355896, Validation MAE = 0.6692285537719727
------------------------------
Training model for stock ID 98
(1, 34, 16)
Threshold = 0.44999999999999996
Training data set size =  (1071, 55, 16)
Validation data set size =  (165, 55, 16)
Training MAE = 0.8948293328285217, Validation MAE = 0.4837295114994049
------------------------------
Training model for stock ID 99
(1, 0, 16)
Threshold = 0.44999999999999996
Training data set size =  (1001, 55, 16)
Validation data set size =  (165, 55, 16)
Training MAE = 0.8247239589691162, Validation MAE = 0.7731720209121704
-------------------------

(1, 16, 16)
Threshold = 0.5499999999999999
Training data set size =  (1005, 55, 16)
Validation data set size =  (165, 55, 16)
Training MAE = 0.8733794689178467, Validation MAE = 0.6231144070625305
------------------------------
Training model for stock ID 129
(1, 43, 16)
Threshold = 0.44999999999999996
Training data set size =  (977, 55, 16)
Validation data set size =  (165, 55, 16)
Training MAE = 0.9453544020652771, Validation MAE = 0.6061013340950012
------------------------------
Training model for stock ID 130
(1, 1, 16)
Threshold = 0.44999999999999996
Training data set size =  (1247, 55, 16)
Validation data set size =  (165, 55, 16)
Training MAE = 0.7928402423858643, Validation MAE = 0.532344400882721
------------------------------
Training model for stock ID 131
(1, 27, 16)
Threshold = 0.44999999999999996
Training data set size =  (666, 55, 16)
Validation data set size =  (165, 55, 16)
Training MAE = 1.0160295963287354, Validation MAE = 0.9178948402404785
------------------------

(1, 0, 16)
Threshold = 0.5499999999999999
Training data set size =  (1249, 55, 16)
Validation data set size =  (165, 55, 16)
Training MAE = 0.8852183222770691, Validation MAE = 0.8880811333656311
------------------------------
Training model for stock ID 161
(1, 48, 16)
Threshold = 0.44999999999999996
Training data set size =  (1408, 55, 16)
Validation data set size =  (165, 55, 16)
Training MAE = 0.9320746064186096, Validation MAE = 0.8165134191513062
------------------------------
Training model for stock ID 162
(1, 0, 16)
Threshold = 0.44999999999999996
Training data set size =  (747, 55, 16)
Validation data set size =  (165, 55, 16)
Training MAE = 0.9202786087989807, Validation MAE = 0.897783100605011
------------------------------
Training model for stock ID 163
(1, 9, 16)
Threshold = 0.5499999999999999
Training data set size =  (1175, 55, 16)
Validation data set size =  (165, 55, 16)
Training MAE = 0.7582001090049744, Validation MAE = 0.4384823739528656
--------------------------

(1, 29, 16)
Threshold = 0.44999999999999996
Training data set size =  (855, 55, 16)
Validation data set size =  (165, 55, 16)
Training MAE = 0.7645357251167297, Validation MAE = 0.44793665409088135
------------------------------
Training model for stock ID 193
(1, 16, 16)
Threshold = 0.5499999999999999
Training data set size =  (1089, 55, 16)
Validation data set size =  (165, 55, 16)
Training MAE = 0.9252426028251648, Validation MAE = 0.7753530144691467
------------------------------
Training model for stock ID 194
(1, 30, 16)
Threshold = 0.44999999999999996
Training data set size =  (918, 55, 16)
Validation data set size =  (165, 55, 16)
Training MAE = 0.8198148608207703, Validation MAE = 0.5134080648422241
------------------------------
Training model for stock ID 195
(1, 63, 16)
Threshold = 0.5499999999999999
Training data set size =  (1133, 55, 16)
Validation data set size =  (165, 55, 16)
Training MAE = 0.8119176030158997, Validation MAE = 0.8312662839889526
----------------------

In [25]:
print('The MAEs of the models are: -')
count1 = 0
count2 = 0
avg_MAE = 0
for curr_stock_id in models_dict.keys():
    print(f'Stock ID = {curr_stock_id}, Cosine similarity threshold = {models_dict[curr_stock_id][1]}, Training MAE = {models_dict[curr_stock_id][2]}, Validation MAE = {models_dict[curr_stock_id][3]}')

    #Check the number of stocks predictions for which have a validation MAE of less than 5.
    if models_dict[curr_stock_id][3] < 4:
        count1 += 1

    if models_dict[curr_stock_id][3] < 5:
        count2 += 1
    
    avg_MAE += models_dict[curr_stock_id][3]

avg_MAE = avg_MAE/len(models_dict)

The MAEs of the models are: -
Stock ID = 0, Cosine similarity threshold = 0.95, Training MAE = 0.7623547911643982, Validation MAE = 0.7081812620162964
Stock ID = 1, Cosine similarity threshold = 0.95, Training MAE = 0.7468001246452332, Validation MAE = 0.6311120986938477
Stock ID = 2, Cosine similarity threshold = 0.95, Training MAE = 0.8455199599266052, Validation MAE = 0.8830938339233398
Stock ID = 3, Cosine similarity threshold = 0.95, Training MAE = 0.7316550612449646, Validation MAE = 0.5541003346443176
Stock ID = 4, Cosine similarity threshold = 0.95, Training MAE = 0.6467030644416809, Validation MAE = 0.5069093704223633
Stock ID = 5, Cosine similarity threshold = 0.95, Training MAE = 0.6203770041465759, Validation MAE = 0.38522112369537354
Stock ID = 6, Cosine similarity threshold = 0.95, Training MAE = 0.6207630634307861, Validation MAE = 0.6588804125785828
Stock ID = 7, Cosine similarity threshold = 0.95, Training MAE = 0.49060478806495667, Validation MAE = 0.38904255628585815

In [26]:
print('Number of stocks predictions for which have a validation MAE of less than 4 = ', count1)
print('Number of stocks predictions for which have a validation MAE of less than 5 = ', count2)
print('Average validation MAE = ', avg_MAE)

Number of stocks predictions for which have a validation MAE of less than 4 =  200
Number of stocks predictions for which have a validation MAE of less than 5 =  200
Average validation MAE =  0.6868573960661888
